In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Our CRAZY machine learning thing :) ...

In [3]:
from nn_model import NNModel
from preprocessing import *
from nn_model import *
import pandas as pd

In [4]:
preprocessing = Preprocessing(use_transformations=False)
y[y==-1] = 0
y = y.astype(np.int)
preprocessing.replace_outliers_by_nan(tX)
num_nan_in_row = np.isnan(tX).sum(axis=1)
mask = (y==0) & (num_nan_in_row>8)
y = y[~mask]
tX = tX[~mask]
num_nan_in_row = np.isnan(tX).sum(axis=1)

In [5]:
lr = 0.01
lambda_= 2
batch_size = 128
epochs = 1
train_data, models = preprocessing.preprocess(tX, lr=lr, lambda_=lambda_, batch_size=batch_size, epochs=epochs)

{}
Training started
>Epoch #1:	[####################]; Loss: 9.93690107423754Training ended
Training started
>Epoch #1:	[####################]; Loss: 9.813370632083137Training ended
Training started
>Epoch #1:	[####################]; Loss: 12.472599495029652Training ended
Training started
>Epoch #1:	[####################]; Loss: 17.419911204053044Training ended
Training started
>Epoch #1:	[####################]; Loss: 10.077063881057654Training ended
Training started
>Epoch #1:	[####################]; Loss: 14.395445381182087Training ended
Training started
>Epoch #1:	[####################]; Loss: 6.3033436638219715Training ended
Training started
>Epoch #1:	[####################]; Loss: 10.304097026407227Training ended
Training started
>Epoch #1:	[####################]; Loss: 7.925523295606021Training ended
Training started
>Epoch #1:	[####################]; Loss: 12.568345735798486Training ended
Training started
>Epoch #1:	[####################]; Loss: 14.158763975604039Training ended


In [6]:
def hyperparamters_tunning(x, y, lrs, lambdas, model_parameters):
    for lr in lrs:
        for lamda_ in lambdas:
            model = NNModel(x.shape[1], loss='logistic_reg')
            for layer_parameters in model_parameters:
                units = layer_parameters['units']
                activation = layer_parameters['activation']
                if activation not in [None, 'softmax', 'sigmoid']:
                    raise Exception("Can't apply activation fuction")
                
                model.add_layer(units=units, activation=activation)
                model.train(x, y, lr=lr, lambda_=lambda_, batch_size=64, epochs=100, verbose=1)

In [7]:
model = NNModel(train_data.shape[1])
model.add_layer(1, activation='sigmoid')

In [8]:
lr = 0.01
lambda_= 10
batch_size = 128
epochs = 10
verbose = 1
model.train(train_data, y, lr=0.01, lambda_=lambda_, batch_size=batch_size, epochs=epochs, verbose=verbose, loss_fun='logistic_reg')

Training started
>Epoch #10:	[####################]; Loss: 0.6423687486468742Training ended


In [9]:
pred = model.predict(train_data)

In [10]:
res = pred>0.5

In [11]:
res = res.squeeze()

In [12]:
pred = np.zeros(y.shape)

In [13]:
pred[res] = 1

In [14]:
np.mean(pred==y)

0.637001008093223

## Generate predictions and save ouput in csv format for submission:

In [15]:
DATA_TEST_PATH = DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/test.csv')
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [18]:
preprocessing = Preprocessing()
lr = 0.01
lambda_= 2
batch_size = 128
epochs = 100
test_data, models = preprocessing.preprocess(tX_test, lr=lr, lambda_=lambda_, batch_size=batch_size, epochs=epochs, train=False, models=models)

{17: <nn_model.NNModel object at 0x7f129f052190>, 44: <nn_model.NNModel object at 0x7f12a11a9690>, 48: <nn_model.NNModel object at 0x7f129f03abd0>, 49: <nn_model.NNModel object at 0x7f12a1287950>, 50: <nn_model.NNModel object at 0x7f12a0ad0fd0>, 56: <nn_model.NNModel object at 0x7f12a11ad450>, 66: <nn_model.NNModel object at 0x7f12b88e0510>, 67: <nn_model.NNModel object at 0x7f12c8212390>, 68: <nn_model.NNModel object at 0x7f129e0b9b90>, 69: <nn_model.NNModel object at 0x7f129e0b9e50>, 70: <nn_model.NNModel object at 0x7f12b879df10>, 71: <nn_model.NNModel object at 0x7f129e0b9fd0>}


In [22]:
OUTPUT_PATH = 'prediction' # TODO: fill in desired name of output file for submission
y_pred = model.predict(test_data)
res = y_pred>0.5
res = res.squeeze()
pred = -np.ones(res.shape)
pred[res] = 1
create_csv_submission(ids_test, pred, OUTPUT_PATH)